## Instructions

You are asked to complete the following files:
* **pruned_layers.py**, which contains the pruning of DNNs to reduce the storage of insignificant weight parameters with 2 methods: pruning by percentage and prune by standard deviation.
* **train_util.py**, which includes the training process of DNNs with pruned connections.
* **quantize.py**, which applies the quantization (weight sharing) part on the DNN to reduce the storage of weight parameters.
* **huffman_coding.py**, which applies the Huffman coding onto the weight of DNNs to further compress the weight size.

You are asked to submit the following files:
* **net_before_pruning.pt**, which is the set of weight parameters before applying pruning on DNN weight parameters.
* **net_after_pruning.pt**, which is the set of weight paramters after applying pruning on DNN weight parameters.
* **net_after_quantization.pt**, which is the set of weight parameters after applying quantization (weight sharing) on DNN weight parameters.
* **codebook_resnet20.npy**, which is the quantization codebook of each layer after applying quantization (weight sharing).
* **huffman_encoding.npy**, which is the encoding map of each item within the quantization codebook in the whole DNN architecture.
* **huffman_freq.npy**, which is the frequency map of each item within the quantization codebook in the whole DNN. 

To ensure fair grading policy, we fix the choice of model to ResNet-20. You may check the implementation in **resnet20.py** for more details.

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune_after_prune, test
from quantize import quantize_whole_model
from huffman_coding import huffman_coding
from summary import summary
import torch
import numpy as np
from prune import prune

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Full-precision model training

In [2]:
net = ResNetCIFAR(num_layers=20)
net = net.to(device)

# Uncomment to load pretrained weights
# net.load_state_dict(torch.load("net_before_pruning.pt"))
# Comment if you have loaded pretrained weights
# train(net, epochs=100, batch_size=128, lr=0.01, reg=1e-2)
# train(net, epochs=100, batch_size=128, lr=0.01, reg=1e-2)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_before_pruning.pt"))
test(net)

# train(net, epochs=10, batch_size=128, lr=0.01, reg=1e-2)

Files already downloaded and verified
Test Loss=0.2586, Test accuracy=0.9168


In [5]:
print("-----Summary before pruning-----")
summary(net)
print("-------------------------------")

-----Summary before pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		218			0.747685
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	4608		1120			0.756944
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	2304		839			0.635851
8		BatchNorm	N/A		N/A			N/A
9		Convolutional	512		143			0.720703
10		BatchNorm	N/A		N/A			N/A
11		ReLU		N/A		N/A			N/A
12		Convolutional	2304		829			0.640191
13		BatchNorm	N/A		N/A			N/A
14		ReLU		N/A		N/A			N/A
15		Convolutional	2304		852			0.630208
16		BatchNorm	N/A		N/A			N/A
17		ReLU		N/A		N/A			N/A
18		Convolutional	2304		806			0.650174
19		BatchNorm	N/A		N/A			N/A
20		ReLU		N/A		N/A			N/A
21		Convolutional	2304		763			0.668837
22		BatchNorm	N/A		N/A			N/A
23		ReLU		N/A		N/A			N/A
24		Convolutional	4608		1754			0.619358
25		BatchNorm	N/A		N/A			N/A
26		ReLU		N/A		N/A			N/A
27		Convolutional	9216		3696			0.598958
28		BatchNorm	N/A		N/A			N/A
29		Convolutional	512		164			0.679688


### Pruning & Finetune with pruned connections

In [6]:
# Test accuracy before fine-tuning
prune(net, method='std', q=0.0, s=0.8)
test(net)
# print(net.sparsity)

Files already downloaded and verified
Test Loss=1.1101, Test accuracy=0.6618


In [7]:
# Uncomment to load pretrained weights
# net.load_state_dict(torch.load("net_after_pruning.pt"))
# Comment if you have loaded pretrained weights
finetune_after_prune(net, epochs=20, batch_size=128, lr=0.00001, reg=1e-3)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.3529	acc=0.9114	3057.0 examples/second
[Step=100]	Loss=0.3596	acc=0.9086	6449.7 examples/second
[Step=150]	Loss=0.3532	acc=0.9110	6622.8 examples/second
[Step=200]	Loss=0.3448	acc=0.9139	6665.6 examples/second
[Step=250]	Loss=0.3394	acc=0.9154	6702.7 examples/second
[Step=300]	Loss=0.3339	acc=0.9176	6727.8 examples/second
[Step=350]	Loss=0.3290	acc=0.9191	6728.8 examples/second
Test Loss=0.3592, Test acc=0.8951
Saving...

Epoch: 1
[Step=400]	Loss=0.2973	acc=0.9280	1822.6 examples/second
[Step=450]	Loss=0.3034	acc=0.9224	6782.2 examples/second
[Step=500]	Loss=0.3030	acc=0.9237	6803.8 examples/second
[Step=550]	Loss=0.3003	acc=0.9258	6668.4 examples/second
[Step=600]	Loss=0.2982	acc=0.9268	6802.9 examples/second
[Step=650]	Loss=0.2959	acc=0.9269	6859.6 examples/second
[Step=700]	Loss=0.2949	acc=0.9270	6793.0 examples/second
[Step=750]	Loss=0.2941	acc=0.9273	6781.9 e

In [8]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_after_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3033, Test accuracy=0.9056


In [10]:
print("-----Summary After pruning-----")
summary(net)
print("-------------------------------")

-----Summary After pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		492			0.430556
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	4608		2633			0.428602
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	2304		1768			0.232639
8		BatchNorm	N/A		N/A			N/A
9		Convolutional	512		313			0.388672
10		BatchNorm	N/A		N/A			N/A
11		ReLU		N/A		N/A			N/A
12		Convolutional	2304		1742			0.243924
13		BatchNorm	N/A		N/A			N/A
14		ReLU		N/A		N/A			N/A
15		Convolutional	2304		1760			0.236111
16		BatchNorm	N/A		N/A			N/A
17		ReLU		N/A		N/A			N/A
18		Convolutional	2304		1637			0.289497
19		BatchNorm	N/A		N/A			N/A
20		ReLU		N/A		N/A			N/A
21		Convolutional	2304		1642			0.287326
22		BatchNorm	N/A		N/A			N/A
23		ReLU		N/A		N/A			N/A
24		Convolutional	4608		3612			0.216146
25		BatchNorm	N/A		N/A			N/A
26		ReLU		N/A		N/A			N/A
27		Convolutional	9216		7338			0.203776
28		BatchNorm	N/A		N/A			N/A
29		Convolutional	512		385			0.248

### Quantization

In [9]:
net.load_state_dict(torch.load("net_after_pruning.pt"))
centers = quantize_whole_model(net, bits=5)
np.save("codebook_resnet20.npy", centers)
torch.save(net.state_dict(), "net_after_quantization.pt")

Complete 1 layers quantization...
Complete 2 layers quantization...
Complete 3 layers quantization...
Complete 4 layers quantization...
Complete 5 layers quantization...
Complete 6 layers quantization...
Complete 7 layers quantization...
Complete 8 layers quantization...
Complete 9 layers quantization...
Complete 10 layers quantization...
Complete 11 layers quantization...
Complete 12 layers quantization...
Complete 13 layers quantization...
Complete 14 layers quantization...
Complete 15 layers quantization...
Complete 16 layers quantization...
Complete 17 layers quantization...
Complete 18 layers quantization...
Complete 19 layers quantization...
Complete 20 layers quantization...
Complete 21 layers quantization...
Complete 22 layers quantization...
Complete 23 layers quantization...


In [5]:
test(net)

Files already downloaded and verified
Test Loss=0.3140, Test accuracy=0.9032


### Huffman Coding

In [10]:
frequency_map, encoding_map ,avbit= huffman_coding(net, centers)
np.save("huffman_encoding", encoding_map)
np.save("huffman_freq", frequency_map)
print(avbit)

3780836: '00101', 0.029059807: '00110', -0.038818043: '001110', 0.040651847: '0011110', 0.060874317: '001111100', 0.07053141: '001111101', 0.0772652: '001111110', 0.09354925: '0011111110', 0.11488338: '0011111111', 0.020194769: '0100', 0.014228703: '0101', -0.012899848: '011', 0.009442024: '1000', -0.028387519: '10010', 0.024220098: '10011', 0.01685347: '1010', -0.019990455: '1011', -0.01021429: '110', -0.016357133: '1110', -0.071333274: '11110000', -0.048703533: '11110001', 0.045993134: '1111001', 0.034784302: '111101', -0.024197405: '11111'}
Average storage for each parameter after Huffman Coding: 4.0839 bits
Complete 2 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
end {0.029612372: '00000', 0.06172983: '0000100', 0.03705186: '0000101', 0.032565698: '000011', -0.01942347: '0001', 0.012650999: '0010', 0.015517792: '0011', -0.025008924: '0100', 0.01933244: '0101', 0.01718993: '0110', 0.014068632: '0111', 0.04896311: '1000000', 0.07565472: '10000010', 0.0

In [11]:
test(net)
summary(net)

Files already downloaded and verified
Test Loss=0.3133, Test accuracy=0.9028
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		218			0.747685
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	4608		1120			0.756944
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	2304		839			0.635851
8		BatchNorm	N/A		N/A			N/A
9		Convolutional	512		143			0.720703
10		BatchNorm	N/A		N/A			N/A
11		ReLU		N/A		N/A			N/A
12		Convolutional	2304		829			0.640191
13		BatchNorm	N/A		N/A			N/A
14		ReLU		N/A		N/A			N/A
15		Convolutional	2304		852			0.630208
16		BatchNorm	N/A		N/A			N/A
17		ReLU		N/A		N/A			N/A
18		Convolutional	2304		806			0.650174
19		BatchNorm	N/A		N/A			N/A
20		ReLU		N/A		N/A			N/A
21		Convolutional	2304		763			0.668837
22		BatchNorm	N/A		N/A			N/A
23		ReLU		N/A		N/A			N/A
24		Convolutional	4608		1754			0.619358
25		BatchNorm	N/A		N/A			N/A
26		ReLU		N/A		N/A			N/A
27		Convolutional	9216		3696			0.598958
28		BatchNorm	N/A		N/A	